# Bank of England technical assessment

- Task I = report on firm prioritisation
- Task II = ML techniques
- Task III = cloud technologies for daily processing

## Task I
Find firms that should be priorotised for investigation. This is based on:

- Firm size (i.e. the biggest firms need more attention) 
- Changing business profile (are firms’ data changing substantially year-on-year?) 
- Outliers from the norm (when looking at a single reporting period, does a firm deviate significantly from the average?) 

### Choose KPIs
#### Size
- **Gross Written Premium (GWP)** – total revenue written by an insurer. Equivalent of turnover for a non-insurance firm. 
  - highlights whether it's a growing business
- **Total assets** - anything that a company owns - highlights it's size
  - total liabilities could indicate support that the company has got through debt to boost growth / operations

Could do scatter plot between these GWP and total assets?

#### Health
- **SCR coverage ratio** – a measure of whether a firm is meeting its prudential capital requirements. Greater than 100% means the firm is holding enough capital to meet the requirement. The size of the buffer (i.e. surplus over 100%) can be important. 
- **(Dervived) NWP / GWP** – suggests stable and effective firm that is able to hold onto a lot of its own risk. It highlights that they're good at underwriting and have confidence in it.
- **Net combined ratio** – (incurred losses plus expenses) / earned premiums. This is a ratio that can indicate the profitability of a firm. If this is less than 100% it indicates a profit. 
- **Gross claims incurred** – a large cost to an insurer. Monitoring how these change over time for a firm is vital. 
- **Excess of Assets over Liabilities** - this is the owners / shareholders ownership, i.e. equity, of the business
  - This can highlight reslience in the firm because they can finance themselves with their own money, and by the same token, absorb lossess in uncertain times. Crucially, they're able to do these two things without borrowing money.
- **Debt to equity ratio** - total liabilites / Excess of assets over liabilities
  - a lower ratio implies less reliance on external borrowing 

# Import data

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

general_cols = ['Firms', 'NWP (£m) ', 'SCR coverage ratio', 'GWP (£m)', 'Total assets (£m)', 'Total liabilities (£m)', 'Excess of assets over liabilities (£m) [= equity]']
insurance_cols = ['Gross claims incurred (£m)', 'Net combined ratio']

df_general = pd.read_excel('data.xlsx', sheet_name='Dataset 1 - General', header=[0,1])
df_insurance = pd.read_excel('data.xlsx', sheet_name='Dataset 2 - Underwriting', header=[0,1])
df_general_drop = df_general.drop(('Firms', 'Firms'), axis=1)
df_insurance_drop = df_insurance.drop(('Firms', 'Firms'), axis=1)

In [2]:
df_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 41 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   (Firms, Firms)                                               325 non-null    object 
 1   (NWP (£m) , 2016YE)                                          325 non-null    float64
 2   (NWP (£m) , 2017YE)                                          325 non-null    float64
 3   (NWP (£m) , 2018YE)                                          325 non-null    float64
 4   (NWP (£m) , 2019YE)                                          325 non-null    float64
 5   (NWP (£m) , 2020YE)                                          325 non-null    float64
 6   (SCR (£m), 2016YE)                                           325 non-null    float64
 7   (SCR (£m), 2017YE)                                           325 non-null    flo

In [3]:
df_insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 46 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   (Firms, Firms)                                          456 non-null    object 
 1   (Gross claims incurred (£m), 2016YE)                    456 non-null    float64
 2   (Gross claims incurred (£m), 2017YE)                    456 non-null    float64
 3   (Gross claims incurred (£m), 2018YE)                    456 non-null    float64
 4   (Gross claims incurred (£m), 2019YE)                    456 non-null    float64
 5   (Gross claims incurred (£m), 2020YE)                    456 non-null    float64
 6   (Gross BEL (inc. TPs as whole, pre-TMTP) (£m), 2016YE)  456 non-null    float64
 7   (Gross BEL (inc. TPs as whole, pre-TMTP) (£m), 2017YE)  456 non-null    float64
 8   (Gross BEL (inc. TPs as whole, pre-TMTP)

# Data cleaning

## Check duplicates

In [4]:
df_insurance.duplicated().sum()

0

In [5]:
df_general.duplicated().sum()

0

## Check for all zeros

In [6]:
df_general_zeros = df_general_drop[(df_general_drop == 0).all(axis=1)]
df_general_zeros

NWP (£m)                              SCR (£m)                       \
       2016YE 2017YE 2018YE 2019YE 2020YE   2016YE 2017YE 2018YE 2019YE   
7         0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
10        0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
28        0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
49        0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
64        0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
67        0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
111       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
115       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
118       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
127       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
161       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
175       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
197       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
205       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
250       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
256       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
258       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
260       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   
309       0.0    0.0    0.0    0.0    0.0      0.0    0.0    0.0    0.0   

            ... Total liabilities (£m)                              \
    2020YE  ...                 2016YE 2017YE 2018YE 2019YE 2020YE   
7      0.0  ...                    0.0    0.0    0.0    0.0    0.0   
10     0.0  ...                    0.0    0.0    0.0    0.0    0.0   
28     0.0  ...                    0.0    0.0    0.0    0.0    0.0   
49     0.0  ...                    0.0    0.0    0.0    0.0    0.0   
64     0.0  ...                    0.0    0.0    0.0    0.0    0.0   
67     0.0  ...                    0.0    0.0    0.0    0.0    0.0   
111    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
115    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
118    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
127    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
161    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
175    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
197    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
205    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
250    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
256    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
258    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
260    0.0  ...                    0.0    0.0    0.0    0.0    0.0   
309    0.0  ...                    0.0    0.0    0.0    0.0    0.0   

    Excess of assets over liabilities (£m) [= equity]                       \
                                               2016YE 2017YE 2018YE 2019YE   
7                                                 0.0    0.0    0.0    0.0   
10                                                0.0    0.0    0.0    0.0   
28                                                0.0    0.0    0.0    0.0   
49                                                0.0    0.0    0.0    0.0   
64                                                0.0    0.0    0.0    0.0   
67                                                0.0    0.0    0.0    0.0   
111                                               0.0    0.0    0.0    0.0   
115                                               0.0    0.0    0.0    0.0   
118                                               0.0    0.0    0.0    0.0   
127                                               0.0    0.0    0.0    0.0   
161                  

In [7]:
df_insurance_zeros = df_insurance_drop[(df_insurance_drop == 0).all(axis=1)]
df_insurance_zeros

Gross claims incurred (£m)                              \
                        2016YE 2017YE 2018YE 2019YE 2020YE   
2                          0.0    0.0    0.0    0.0    0.0   
6                          0.0    0.0    0.0    0.0    0.0   
12                         0.0    0.0    0.0    0.0    0.0   
14                         0.0    0.0    0.0    0.0    0.0   
15                         0.0    0.0    0.0    0.0    0.0   
..                         ...    ...    ...    ...    ...   
443                        0.0    0.0    0.0    0.0    0.0   
445                        0.0    0.0    0.0    0.0    0.0   
447                        0.0    0.0    0.0    0.0    0.0   
451                        0.0    0.0    0.0    0.0    0.0   
455                        0.0    0.0    0.0    0.0    0.0   

    Gross BEL (inc. TPs as whole, pre-TMTP) (£m)                              \
                                          2016YE 2017YE 2018YE 2019YE 2020YE   
2                                            0.0    0.0    0.0    0.0    0.0   
6                                            0.0    0.0    0.0    0.0    0.0   
12                                           0.0    0.0    0.0    0.0    0.0   
14                                           0.0    0.0    0.0    0.0    0.0   
15                                           0.0    0.0    0.0    0.0    0.0   
..                                           ...    ...    ...    ...    ...   
443                                          0.0    0.0    0.0    0.0    0.0   
445                                          0.0    0.0    0.0    0.0    0.0   
447                                          0.0    0.0    0.0    0.0    0.0   
451                                          0.0    0.0    0.0    0.0    0.0   
455                                          0.0    0.0    0.0    0.0    0.0   

     ... Gross expense ratio                             Gross combined ratio  \
     ...              2016YE 2017YE 2018YE 2019YE 2020YE               2016YE   
2    ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
6    ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
12   ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
14   ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
15   ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
..   ...                 ...    ...    ...    ...    ...                  ...   
443  ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
445  ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
447  ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
451  ...                 0.0    0.0    0.0    0.0    0.0                  0.0   
455  ...                 0.0    0.0    0.0    0.0    0.0                  0.0   

                                 
    2017YE 2018YE 2019YE 2020YE  
2      0.0    0.0    0.0    0.0  
6      0.0    0.0    0.0    0.0  
12     0.0    0.0    0.0    0.0  
14     0.0    0.0    0.0    0.0  
15     0.0    0.0    0.0    0.0  
..     ...    ...    ...    ...  
443    0.0    0.0    0.0    0.0  
445    0.0    0.0    0.0    0.0  
447    0.0    0.0    0.0    0.0  
451    0.0    0.0    0.0    0.0  
455    0.0    0.0    0.0    0.0  

[116 rows x 45 columns]

### Remove zero rows from each dataframe then merge

In [8]:
df_general = df_general.drop(df_general_zeros.index)
df_insurance = df_insurance.drop(df_insurance_zeros.index)

In [9]:
df_merged = pd.merge(df_general, df_insurance, how='left', left_on=df_general[('Firms', 'Firms')], right_on=df_insurance[('Firms', 'Firms')])
df_merged = df_merged.drop(('key_0', ''), axis=1)
df_merged

Firms_x     NWP (£m)                                             \
      Firms_x        2016YE        2017YE        2018YE        2019YE   
0      Firm 1 -17754.100486      0.000000      0.000000      0.000000   
1      Firm 2      9.244369      0.098771     40.812426     35.643071   
2      Firm 3      0.000000      0.806130     36.620199     86.630018   
3      Firm 4  19677.453353  41072.816156  20885.506445  14719.672914   
4      Firm 5    128.812208     10.407371     79.579663     79.423623   
..        ...           ...           ...           ...           ...   
301  Firm 321      0.000000      0.000000     -0.167178     -6.813367   
302  Firm 322   3315.466643   4136.476625   2192.750865   2252.917257   
303  Firm 323      0.000000      0.000000      0.000000      0.000000   
304  Firm 324     29.811172     21.003008     13.551740      6.145755   
305  Firm 325    108.471899    496.296030    437.217641    120.652291   

                       SCR (£m)                                            \
           2020YE        2016YE        2017YE        2018YE        2019YE   
0        0.000000   2021.321862  6.940043e-09      0.000000      0.000000   
1       13.626145     16.459756  1.500907e+01     14.227214     15.037066   
2       54.744180    486.887796  4.957863e+02    626.612217     81.242579   
3    49204.465136  15975.652771  1.536789e+04  10797.123319  22788.359468   
4       61.727295     32.742279  4.395404e+01     31.622572     30.119285   
..            ...           ...           ...           ...           ...   
301     31.258806      0.000000  3.652140e-01     70.399156     68.655722   
302   2780.628432    578.238997  1.926540e+03   1366.552217    849.073657   
303      0.000000      6.459131  2.803774e+01      5.689804      1.674462   
304     49.793157     45.967315  5.732320e+01     36.910810      1.012305   
305      0.000000    402.002179  4.082730e+02    477.423649    346.976664   

     ... Gross expense ratio                                           \
     ...              2016YE     2017YE    2018YE    2019YE    2020YE   
0    ...            0.000000  77.251254  0.000000  0.000000  0.000000   
1    ...            0.391179   0.685941  1.433284  0.000000  0.000000   
2    ...                 NaN        NaN       NaN       NaN       NaN   
3    ...            0.231139   0.285471  0.054150  0.051848 -0.162883   
4    ...            0.236128   0.232725  0.204508  0.131739  0.048703   
..   ...                 ...        ...       ...       ...       ...   
301  ...            0.296448   0.191321  0.032038  0.396320  0.227326   
302  ...            0.499785   0.323232  0.753450  0.390250  0.513934   
303  ...                 NaN        NaN       NaN       NaN       NaN   
304  ...            0.166180   0.328329  0.278285  0.238669  0.228917   
305  ...            0.012621   0.039664  0.029074  0.007590  0.045571   

    Gross combined ratio                                          
                  2016YE    2017YE    2018YE    2019YE    2020YE  
0               0.000000  0.675051  0.000000  0.000000  0.000000  
1               1.750946  1.321523  1.545325  0.000000  0.000000  
2                    NaN       NaN       NaN       NaN       NaN  
3               1.047234  0.476384  3.307379  1.247403 -2.303093  
4               0.243611  1.977732  2.495287  0.365455  1.133645  
..                   ...       ...       ...       ...       ...  
301             0.839221  1.814716  0.344145  0.872099  1.164540  
302             0.063623  0.761518  0.998770  1.064060  1.984687  
303                  NaN       NaN       NaN       NaN       NaN  
304             1.812040  1.868333  0.800207  0.420591  1.515067  
305             0.077135  0.009230  0.724871  0.430852  0.157979  

[306 rows x 87 columns]

# Filter by size

First remove firms with GWP equal to or less than zero

In [10]:
df_merged[df_merged['GWP (£m)'].sum(axis=1) <= 0]

Firms_x NWP (£m)                                               SCR (£m)  \
      Firms_x    2016YE    2017YE    2018YE    2019YE    2020YE        2016YE   
9     Firm 12  0.000000  0.000000  0.000000  0.000000  0.000000  1.126037e+00   
13    Firm 16  0.000000  0.000000  0.000000  0.000000  0.000000  7.158603e+00   
15    Firm 18  0.000000  0.000000  0.000000  0.000000  0.000000  3.732730e-01   
18    Firm 21  0.000000  0.000000  0.000000  0.000000  0.027259  3.507516e-01   
27    Firm 31  0.001498  0.000000  0.000000  0.000000  0.000000  1.605136e+00   
39    Firm 43  0.000000  0.000000  0.000000  0.000000  0.000000  3.620191e+00   
51    Firm 56 -0.051840  0.000000  0.000000  0.000000  0.000000  1.405326e+00   
56    Firm 61  0.838222 -0.663091 -4.750917 -0.271076  0.653909  2.058780e+02   
76    Firm 83  0.000000  0.000000  0.000000  0.000000  0.000000  3.451869e+01   
77    Firm 84  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000e+00   
78    Firm 85  0.000000  0.000000  0.000000  0.000000  0.000000  1.977941e+01   
87    Firm 94  0.000000  0.000000  0.000000  0.000000  0.000000  1.067017e-02   
89    Firm 96 -0.019008  0.003815  0.000000  0.000000  0.000000  1.036972e+01   
91    Firm 98  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000e+00   
95   Firm 102  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000e+00   
96   Firm 103  0.000000  0.000000  0.000000  0.000000  0.000000  1.005624e+00   
101  Firm 108  0.000000  0.000000  0.000000  0.000000  0.000000  2.658587e-01   
102  Firm 109  0.000000  0.000000  0.000000  0.000000  0.000000  2.582207e+01   
112  Firm 122  0.000000  0.000000  0.000000  0.000000  0.000000  4.386420e-01   
117  Firm 127  0.000000  0.000000  0.000000  0.000000  0.000000  1.121213e-07   
118  Firm 129  0.000000  0.000000  0.000000  0.000000  0.000000  1.679018e+01   
124  Firm 135  0.000000  0.000000  0.000000  0.000000  0.000000  1.808554e+01   
130  Firm 141  0.000000  0.000000  0.000000  0.000000  0.000000  8.441410e+01   
134  Firm 145  0.000000  0.000000  0.000000  0.000000  0.000000  2.142746e+00   
136  Firm 147  0.000000  0.000000  0.000000  0.000000  0.000000  4.607096e+00   
156  Firm 168  0.000000  0.000000  0.000000  0.000000  0.000000  8.967226e-01   
160  Firm 172  0.000000  0.000000  0.000000  0.000000  0.000000  9.079356e+00   
177  Firm 190  0.000000  0.000000  0.000000  0.000000  0.000000  9.364743e-01   
192  Firm 207 -0.963861 -1.390090  0.000000  0.000000  0.000000  4.511842e+01   
202  Firm 217  0.000000  0.000000  0.000000  0.000000  0.000000  2.492465e+01   
209  Firm 224  0.000000  0.000000  0.000000  0.000000  0.000000  5.146415e-01   
210  Firm 225 -0.037127  0.000098  0.004784  0.000000  0.000000  2.304387e+00   
217  Firm 232  0.000000  0.000000  0.000000  0.000000  0.000000  7.808473e-02   
218  Firm 233  0.000000  0.000000  0.000000  0.000000  0.166024  3.339310e+00   
223  Firm 238  0.000000  0.000000  0.000000  0.000000  0.000000  4.862705e-01   
224  Firm 239  0.000000  0.000000  0.000000  0.000000  0.000000  3.101172e+00   
234  Firm 249  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000e+00   
240  Firm 256  0.000000  0.000000  0.000000  0.000000  0.000000  7.235792e+00   
250  Firm 269  0.000000  0.000000  0.000000  0.000000  0.000000  2.130992e+01   
251  Firm 270  0.000000  0.000000  0.000000  0.000000  0.000000  3.781871e-01   
252  Firm 271  0.000000  0.000000  0.000000  0.000000  0.000000  2.025565e+00   
258  Firm 277 -0.438653 -0.029825 -0.157063 -0.033910  0.000000  3.907264e-01   
272  Firm 291  0.000000  0.000000  0.000000  0.000000  0.000000  1.209135e-01   
281  Firm 300  0.000000  0.000000  0.000000  0.000000  0.000000  1.254419e+00   
292  Firm 312  0.000000  0.000000  0.000000  0.000000  0.000000  3.904599e-01   
294  Firm 314  0.000000  0.000000  0.000000  0.000000  0.000000  3.303004e-01   
300  Firm 320  0.000000  0.000000  0.000000  0.000000  0.000000  1.412290e-06   
303  Firm 323  0.000000  0.000000  0.

In [11]:
df_merged = df_merged[df_merged['GWP (£m)'].sum(axis=1) > 0]

## PRA GWP threshold pre 2023 = £5M
How many firms does this include? 

Rouhgly 187 firms, after taking the median GWP across all years - too many firms

In [12]:
df_merged[df_merged['GWP (£m)'].median(axis=1) > 5]

Firms_x     NWP (£m)                                             \
      Firms_x        2016YE        2017YE        2018YE        2019YE   
1      Firm 2      9.244369      0.098771     40.812426     35.643071   
2      Firm 3      0.000000      0.806130     36.620199     86.630018   
3      Firm 4  19677.453353  41072.816156  20885.506445  14719.672914   
4      Firm 5    128.812208     10.407371     79.579663     79.423623   
5      Firm 6   3091.392118   3357.934035   3215.472967     53.217589   
..        ...           ...           ...           ...           ...   
297  Firm 317   1635.740935   4105.052125   4838.904507   4451.743449   
298  Firm 318    102.017050     74.829159     83.869961    310.067014   
302  Firm 322   3315.466643   4136.476625   2192.750865   2252.917257   
304  Firm 324     29.811172     21.003008     13.551740      6.145755   
305  Firm 325    108.471899    496.296030    437.217641    120.652291   

                       SCR (£m)                                            \
           2020YE        2016YE        2017YE        2018YE        2019YE   
1       13.626145     16.459756     15.009073     14.227214     15.037066   
2       54.744180    486.887796    495.786299    626.612217     81.242579   
3    49204.465136  15975.652771  15367.887099  10797.123319  22788.359468   
4       61.727295     32.742279     43.954039     31.622572     30.119285   
5     5969.707516   5127.365839   5252.809367   5460.800869   4581.916567   
..            ...           ...           ...           ...           ...   
297   3479.287464     57.943592    221.532471     99.388160    131.898114   
298      0.000000    472.640615    609.931305    119.423876    274.780500   
302   2780.628432    578.238997   1926.540266   1366.552217    849.073657   
304     49.793157     45.967315     57.323200     36.910810      1.012305   
305      0.000000    402.002179    408.272986    477.423649    346.976664   

     ... Gross expense ratio                                          \
     ...              2016YE    2017YE    2018YE    2019YE    2020YE   
1    ...            0.391179  0.685941  1.433284  0.000000  0.000000   
2    ...                 NaN       NaN       NaN       NaN       NaN   
3    ...            0.231139  0.285471  0.054150  0.051848 -0.162883   
4    ...            0.236128  0.232725  0.204508  0.131739  0.048703   
5    ...            0.105267  0.451178  0.239576  0.261323  0.065710   
..   ...                 ...       ...       ...       ...       ...   
297  ...            0.481435  0.369913  0.042752  0.097238  0.060318   
298  ...            0.416867  0.043522  1.018493  1.105734  0.859473   
302  ...            0.499785  0.323232  0.753450  0.390250  0.513934   
304  ...            0.166180  0.328329  0.278285  0.238669  0.228917   
305  ...            0.012621  0.039664  0.029074  0.007590  0.045571   

    Gross combined ratio                                          
                  2016YE    2017YE    2018YE    2019YE    2020YE  
1               1.750946  1.321523  1.545325  0.000000  0.000000  
2                    NaN       NaN       NaN       NaN       NaN  
3               1.047234  0.476384  3.307379  1.247403 -2.303093  
4               0.243611  1.977732  2.495287  0.365455  1.133645  
5               0.650369  0.904271  0.147146  0.351034  0.687860  
..                   ...       ...       ...       ...       ...  
297             0.262281  0.077431  1.742934  0.519823  0.512366  
298             0.174716 -0.115464  1.470637  0.218232  0.558187  
302             0.063623  0.761518  0.998770  1.064060  1.984687  
304             1.812040  1.868333  0.800207  0.420591  1.515067  
305             0.077135  0.009230  0.724871  0.430852  0.157979  

[187 rows x 87 columns]

## Get top decile

In [13]:
# Create an array of percentiles
percentiles_16_20 = np.array(range(10, 100, 10))

# Calculate deciles
deciles_16_20 = np.percentile(df_merged['GWP (£m)'].sum(axis=1).values, percentiles_16_20)

# Create a bar plot of the decile counts
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=percentiles_16_20,
        y=deciles_16_20,
        name='GWP (£m)'
    )
)

We will first filter the firms to get the top 10% of firms in terms of size.

This equates to a total GWP value of over 10,000 million, i.e. 10 billion, over the 2016-2022 period.

## top 10 percentile firms

In [14]:
largest_firms_16_20 = df_merged[df_merged['GWP (£m)'].sum(axis=1) > deciles_16_20[-1]]
largest_firms_16_20

Firms_x     NWP (£m)                                             \
      Firms_x        2016YE        2017YE        2018YE        2019YE   
3      Firm 4  19677.453353  41072.816156  20885.506445  14719.672914   
5      Firm 6   3091.392118   3357.934035   3215.472967     53.217589   
6      Firm 7   3148.478356   3307.129485    713.846420  18232.376510   
8     Firm 10    404.116022   3882.087306   6640.433866   5196.618355   
14    Firm 17   5210.936051   1146.930176   1620.223253   1256.934295   
22    Firm 25   9622.277578   2356.626073   1227.714013   7134.648741   
23    Firm 26  75526.673293   2187.587977  11251.347661   6683.963247   
26    Firm 30    198.455726   5260.728479   9063.121471   6124.600246   
30    Firm 34  10135.258636   3630.292355   3530.844212   -184.091940   
34    Firm 38   6060.193694   3345.293341   1666.572447      0.000000   
46    Firm 51      0.000000    427.488826   3812.031443   5655.158598   
66    Firm 73   8118.728208    880.739085   6813.491255  11752.057330   
94   Firm 101   7932.403100   3304.202645   2397.454118   4804.131318   
97   Firm 104     12.743263     63.765439  18870.464612   1697.619649   
98   Firm 105  41465.110343  18539.933748    -11.983526  21335.956931   
100  Firm 107   4240.167742    279.984489   3969.084444   1213.256492   
110  Firm 120    534.840652   2406.779146   6902.885913   4542.235555   
140  Firm 151  -1578.071724  10665.118064  12231.905984   7970.539382   
185  Firm 199      1.217636  10050.601335  11509.508404   3395.026046   
195  Firm 210  30031.048551  19261.406692  41591.397560  25473.307182   
219  Firm 234   3700.803327   5344.309755   1155.111534   3998.978113   
232  Firm 247   2489.220023  25897.338404   6237.267910   6512.232956   
257  Firm 276     -0.003989   9897.767779   2565.770457   1179.993877   
282  Firm 301  -1123.890314    325.967010    615.719690   1962.978284   
291  Firm 311    -15.161236  14897.664119  12734.766714    381.198271   
297  Firm 317   1635.740935   4105.052125   4838.904507   4451.743449   

                       SCR (£m)                                            \
           2020YE        2016YE        2017YE        2018YE        2019YE   
3    49204.465136  15975.652771  15367.887099  10797.123319  22788.359468   
5     5969.707516   5127.365839   5252.809367   5460.800869   4581.916567   
6     6486.658898   3352.500439   2155.201622   4246.239082   7597.763417   
8      221.986516   3839.041694   5102.458601   2578.827206    323.481575   
14       0.000000    173.807351    153.049331    136.491987     12.368766   
22    9765.612912     11.952215     10.593245      2.196166      1.771300   
23   16395.671694   4282.790194   3851.999871   1061.571588     98.028981   
26    3396.242957   2801.422895   3545.731490   3476.431962   1392.059988   
30    6105.500028   8785.961467   3011.610527   5924.159038   3884.820732   
34       0.000000   3315.022557   4255.464361   3786.532326      0.000000   
46    3789.882711      9.381618     12.507525      8.730373      3.953666   
66    4073.346848   2801.479556   2877.728511    114.351700   1482.493976   
94       0.000000    438.929248   7521.082011   7134.264916   3369.613372   
97    6174.582162      2.618417      0.009747      3.089429     12.813155   
98    2612.553473  18258.886129  17563.273844   4194.659090  16716.945095   
100   2626.465219   1430.846691   2461.859809    134.101122   2123.074932   
110    687.591337   1670.773502    767.585517    237.708483   1477.826751   
140   5554.254802    777.618407   1636.952736    335.459445   1468.616696   
185  13133.461750   1758.634428   4068.120993    565.326745   1934.741620   
195  60699.953265    381.753797    236.783571    270.236997    345.806646   
219   1569.098440   2540.583338   3375.307912   2125.732382   2635.938596   
232    191.590179     76.521717     56.920352     18.731132      1.019865   
257   3073.774783    131.025364    310.558627     89.452935    150.299525   
282   4643.040629    868.370592    8

### Do the firms have all the data we're interested in?

In [15]:
cols = [
    'SCR coverage ratio',
    'NWP (£m) ',
    'GWP (£m)',
    'Net combined ratio',
    'Gross claims incurred (£m)',
    'Excess of assets over liabilities (£m) [= equity]',
    'Total liabilities (£m)',
]

data = {}
for col in cols:
    data[col] = largest_firms_16_20[col].median(axis=1).values

largest_firms_20 = pd.DataFrame(data=data, index=largest_firms_16_20['Firms_x']['Firms_x'].values)
# largest_firms_20_subset = largest_firms_20_subset.fillna(0) # REPLACE WITH 2016-2019 MEAN!!!!
largest_firms_20

,SCR coverage ratio,NWP (£m),GWP (£m),Net combined ratio,Gross claims incurred (£m),Excess of assets over liabilities (£m) [= equity],Total liabilities (£m)
Firm 4,1.224834,20885.506445,41081.453449,1.841528,42.280402,12910.346383,68145.069079
Firm 6,1.104387,3215.472967,4061.617189,0.496601,268.527873,4848.816357,29794.425246
Firm 7,1.649018,3307.129485,5814.003842,NaN,NaN,11434.723658,9079.892449
Firm 10,1.112032,3882.087306,9002.108895,0.795576,122.984554,7807.061167,193831.906382
Firm 17,1.278596,1256.934295,2037.685268,0.439327,1420.733285,89.061371,8108.999581
Firm 25,3.155166,7134.648741,4169.962819,0.504087,1146.991043,23.885705,30445.542807
Firm 26,1.794204,11251.347661,10352.250388,0.973190,315.841298,7266.408767,68511.986170
Firm 30,1.906925,5260.728479,2301.225209,0.000000,0.000000,4611.238915,70632.721322
Firm 34,2.602542,3630.292355,8107.790888,1.610048,5.663691,11528.901477,167736.855249
Firm 38,1.338800,1666.572447,4720.203774,1.379684,360.052219,2531.593485,5844.001778


The below firms do not have complete data for Net combined ratio or Gross claims icurred. This is likely a reporting error. Therefore we will remove them as only four values

In [16]:
largest_firms_20[largest_firms_20.isna().any(axis=1)]

,SCR coverage ratio,NWP (£m),GWP (£m),Net combined ratio,Gross claims incurred (£m),Excess of assets over liabilities (£m) [= equity],Total liabilities (£m)
Firm 7,1.649018,3307.129485,5814.003842,NaN,NaN,11434.723658,9079.892449
Firm 101,1.407634,3304.202645,4418.482952,NaN,NaN,10807.461907,161113.140524
Firm 276,1.474873,2565.770457,5385.561561,NaN,NaN,257.563878,44395.668883
Firm 301,1.078399,615.719690,2339.220517,NaN,NaN,1068.302949,10750.087353


In [17]:
largest_firms_20 = largest_firms_20.dropna()
largest_firms_20

,SCR coverage ratio,NWP (£m),GWP (£m),Net combined ratio,Gross claims incurred (£m),Excess of assets over liabilities (£m) [= equity],Total liabilities (£m)
Firm 4,1.224834,20885.506445,41081.453449,1.841528,42.280402,12910.346383,68145.069079
Firm 6,1.104387,3215.472967,4061.617189,0.496601,268.527873,4848.816357,29794.425246
Firm 10,1.112032,3882.087306,9002.108895,0.795576,122.984554,7807.061167,193831.906382
Firm 17,1.278596,1256.934295,2037.685268,0.439327,1420.733285,89.061371,8108.999581
Firm 25,3.155166,7134.648741,4169.962819,0.504087,1146.991043,23.885705,30445.542807
Firm 26,1.794204,11251.347661,10352.250388,0.973190,315.841298,7266.408767,68511.986170
Firm 30,1.906925,5260.728479,2301.225209,0.000000,0.000000,4611.238915,70632.721322
Firm 34,2.602542,3630.292355,8107.790888,1.610048,5.663691,11528.901477,167736.855249
Firm 38,1.338800,1666.572447,4720.203774,1.379684,360.052219,2531.593485,5844.001778
Firm 51,0.882185,3789.882711,2524.616078,0.000000,0.000000,8.798237,13789.810771


### Visualise largest firms based on size

#### median size

In [18]:
fig = go.Figure()

for col in largest_firms_16_20['GWP (£m)'].columns:
    fig.add_trace(
        go.Box(
            y=largest_firms_16_20['GWP (£m)'][col],
            name=col,
            boxmean=True
        )
    )

fig.add_trace(
        go.Box(
            y=largest_firms_16_20['GWP (£m)'].median(axis=1),
            name='2016-2020 median',
            boxmean=True
        )
    )

fig.update_layout(
    title='Largest firms',
    yaxis_title='Amount (£m)'
)

fig.show()

#### over time

In [19]:
gwp = largest_firms_16_20[['Firms_x','GWP (£m)']].droplevel(0, axis=1).T
gwp.columns = gwp.iloc[0]
gwp = gwp.drop('Firms_x')

fig = go.Figure()

for col in gwp.columns:
    fig.add_trace(
        go.Scatter(
            x=[2016, 2017, 2018, 2019, 2020],
            y=gwp[col],
            name=col,
            mode='lines',
            line=dict(width=2)
        )
    )

fig.update_layout(
    title='GWP (£m)',
    xaxis_title='Year',
    # xaxis_range=[2016, 2020],
    # yaxis_range=[0, 4],
    yaxis_title='GWP (£m)',
    width=1200,
    height=500
)

fig.show()

# Assessing financial stability

## Composite rank analysis

In [20]:
cols = [
    'SCR coverage ratio',
    'NWP (£m) ',
    'GWP (£m)',
    'Net combined ratio',
    'Gross claims incurred (£m)',
    'Excess of assets over liabilities (£m) [= equity]',
    'Total liabilities (£m)',
]

scaler = MinMaxScaler()
data = {}
for col in cols:
    data[col] = largest_firms_16_20[col].mean(axis=1).values

largest_firms_20 = pd.DataFrame(data=data, index=largest_firms_16_20['Firms_x']['Firms_x'].values)
largest_firms_20 = largest_firms_20.fillna(0) # REPLACE WITH 2016-2019 MEAN!!!!
largest_firms_20['NWP/GWP'] = largest_firms_20['NWP (£m) '] / largest_firms_20['GWP (£m)']
largest_firms_20['debt-equity-ratio'] = largest_firms_20['Total liabilities (£m)'] / largest_firms_20['Excess of assets over liabilities (£m) [= equity]']
largest_firms_20 = largest_firms_20.drop(['NWP (£m) ', 'GWP (£m)', 'Total liabilities (£m)'], axis=1)

# min max scaling
data_norm = {}
for col in largest_firms_20.columns:
    # where high values are good
    if col in ['SCR coverage ratio', 'Excess of assets over liabilities (£m) [= equity]', 'NWP/GWP']:
        data_norm[col] = largest_firms_20[col].apply(lambda x: (x - largest_firms_20[col].min()) / (largest_firms_20[col].max() - largest_firms_20[col].min())).values

    # where low values are good
    elif col in ['Net combined ratio', 'Gross claims incurred (£m)', 'debt-equity-ratio']:
        data_norm[col] = largest_firms_20[col].apply(lambda x: (largest_firms_20[col].max() - x) / (largest_firms_20[col].max() - largest_firms_20[col].min())).values

largest_firms_20_norm = pd.DataFrame(data=data_norm, index=largest_firms_20.index)
largest_firms_20_norm['composite_score'] = (
    (largest_firms_20_norm['SCR coverage ratio'] * 0.5) + 
    (largest_firms_20_norm['Net combined ratio'] * 0.1) +
    (largest_firms_20_norm['Gross claims incurred (£m)'] * 0.1) +
    (largest_firms_20_norm['Excess of assets over liabilities (£m) [= equity]'] * 0.1) +
    (largest_firms_20_norm['NWP/GWP'] * 0.1) +
    (largest_firms_20_norm['debt-equity-ratio'] * 0.1)
)
largest_firms_20_norm = largest_firms_20_norm.sort_values('composite_score', ascending=True)
largest_firms_20_norm

,SCR coverage ratio,Net combined ratio,Gross claims incurred (£m),Excess of assets over liabilities (£m) [= equity],NWP/GWP,debt-equity-ratio,composite_score
Firm 51,0.018158,0.000000,0.990807,0.000000,0.294252,0.072412,0.144826
Firm 17,0.000000,0.505285,0.456455,0.006263,0.219615,0.973066,0.216068
Firm 38,0.032992,0.388403,0.879854,0.099861,0.095277,0.999912,0.262827
Firm 210,0.092762,0.541952,0.835101,0.027133,0.125028,0.715030,0.270805
Firm 10,0.036851,0.503175,0.951878,0.350604,0.043322,0.985546,0.301878
Firm 301,0.032247,0.760366,0.990953,0.066901,0.051786,0.996488,0.302773
Firm 311,0.046134,0.515018,0.988145,0.300263,0.025871,0.974143,0.303411
Firm 234,0.133604,0.698559,0.340815,0.091562,0.261131,0.997318,0.305740
Firm 107,0.052564,0.548809,0.970956,0.124765,0.157861,1.000000,0.306521
Firm 120,0.006819,0.760366,0.990948,0.067632,0.324593,0.977039,0.315467


get lowest five ranked firms

In [21]:
worst_five = list(largest_firms_20_norm.index[:5])
worst_five

['Firm 51', 'Firm 17', 'Firm 38', 'Firm 210', 'Firm 10']

add composite score value back to original data

In [22]:
largest_firms_20 = pd.merge(left=largest_firms_20, right=largest_firms_20_norm[['composite_score']], how='left', left_on=largest_firms_20.index, right_on=largest_firms_20_norm.index)
largest_firms_20 = largest_firms_20.rename(columns={'key_0': 'Firms'})
largest_firms_20 = largest_firms_20.sort_values('composite_score', ascending=True)
largest_firms_20 = largest_firms_20.set_index('Firms')
largest_firms_20 = largest_firms_20.rename_axis(None)
largest_firms_20.head()

,SCR coverage ratio,Net combined ratio,Gross claims incurred (£m),Excess of assets over liabilities (£m) [= equity],NWP/GWP,debt-equity-ratio,composite_score
Firm 51,0.973635,2.377042,0.412983,9.936573,0.953950,1387.795002,0.144826
Firm 17,0.833952,0.797427,1519.991917,139.974490,0.827021,42.525131,0.216068
Firm 38,1.087744,1.162823,315.939660,2083.197348,0.615567,2.425468,0.262827
Firm 210,1.547534,0.682800,443.206819,573.266443,0.666163,427.943241,0.270805
Firm 10,1.117433,0.804023,111.119240,7289.002534,0.527211,23.883245,0.301878


## Plan is to take the top five/ten of these metrics and add them to charts to show where the end up!

What are we looking for in our analysis:
- Solvency position
  - SCR ratio coverage vs owners equity - this will highlight firms how well a firm can weather losses?
  - Owner's Equity vs. Gross Claims Incurred
  - Solvency Capital Ratio Coverage Ratio vs. Debt-Equity Ratio
- Risk exposure
  - Gross Claims incurred vs Net combined ratio - shows how higher claims might have impacted profits, if the number of claims is high and the profit is low or negative this would indicate poor underwriting standards.
- going through existing insolvency?
  - see if any firms have very low or no assets in 2020
- volatility in claims as well as net combined ratio (profit), just do a visual analysis.

In [23]:
largest_firms_20_norm.columns

Index(['SCR coverage ratio', 'Net combined ratio',
       'Gross claims incurred (£m)',
       'Excess of assets over liabilities (£m) [= equity]', 'NWP/GWP',
       'debt-equity-ratio', 'composite_score'],
      dtype='object')

### Solvency position

#### SCR ratio coverage vs owners equity
The worst identified firms have low SCR coverage ratio as well as owners equity

In [25]:
fig = go.Figure()

x = 'SCR coverage ratio'
y = 'Excess of assets over liabilities (£m) [= equity]'

fig.add_trace(
    go.Scatter(
        x=largest_firms_20[x],
        y=largest_firms_20[y],
        mode='markers',
        marker=dict(size=16, color=largest_firms_20['composite_score'], colorscale='RdBu', showscale=True),
        showlegend=False,
        hovertext=largest_firms_20.index
    )
)

fig.add_trace(
    go.Scatter(
        x=largest_firms_20[x][:5],
        y=largest_firms_20[y][:5],
        mode='markers',
        marker=dict(size=8, color='yellow', opacity=0.8, symbol='star'),
        showlegend=False,
        hovertext=largest_firms_20.index
    )
)

# format figure to make it look nice
fig.update_layout(
    template='plotly_white',
    font=dict(size=16),
    xaxis_title='SCR coverage ratio',
    yaxis_title="Owner's equity (£m)",
    width=1000,
    height=500
)
fig.show()
fig.write_image('coverage ratio vs owners equity.png', width=1000, height=500)


#### Owner's Equity vs. Gross Claims Incurred

In [411]:
fig = go.Figure()

x = 'Excess of assets over liabilities (£m) [= equity]'
y = 'Gross claims incurred (£m)'

fig.add_trace(
    go.Scatter(
        x=largest_firms_20[x],
        y=largest_firms_20[y],
        mode='markers',
        marker=dict(size=16, color=largest_firms_20['composite_score'], colorscale='RdBu', showscale=True),
        showlegend=False,
        hovertext=largest_firms_20.index
    )
)

fig.add_trace(
    go.Scatter(
        x=largest_firms_20[x][:5],
        y=largest_firms_20[y][:5],
        mode='markers',
        marker=dict(size=8, color='yellow', opacity=0.8, symbol='star'),
        showlegend=False,
        hovertext=largest_firms_20.index
    )
)

# format figure to make it look nice
fig.update_layout(
    xaxis_title=x,
    yaxis_title=y,
    width=1000,
    height=500
)
fig.show()

#### Solvency Capital Ratio Coverage Ratio vs. Debt-Equity Ratio

Three of the five identified firms have low scr coverage ratio and a high level debt-to-equity ratio. This highlights that a double edged concern of not enough regulatory capital reserved and higher financial obligations from substantial debt.

In [416]:
fig = go.Figure()

x = 'SCR coverage ratio'
y = 'debt-equity-ratio'

fig.add_trace(
    go.Scatter(
        x=largest_firms_20[x],
        y=largest_firms_20[y],
        mode='markers',
        marker=dict(size=16, color=largest_firms_20['composite_score'], colorscale='RdBu', showscale=True),
        showlegend=False,
        hovertext=largest_firms_20.index
    )
)

fig.add_trace(
    go.Scatter(
        x=largest_firms_20[x][:5],
        y=largest_firms_20[y][:5],
        mode='markers',
        marker=dict(size=8, color='yellow', opacity=0.8, symbol='star'),
        showlegend=False,
        hovertext=largest_firms_20.index
    )
)

# format figure to make it look nice
fig.update_layout(
    xaxis_title=x,
    yaxis_title=y,
    width=1000,
    height=500,
    yaxis_type='log',
)
fig.show()

### Risk exposure

#### Gross Claims incurred vs Net combined ratio

In [26]:
fig = go.Figure()

x = 'Gross claims incurred (£m)'
y = 'Net combined ratio'

fig.add_trace(
    go.Scatter(
        x=np.linspace(-10, 3000, len(largest_firms_20)),
        y=[1] * len(largest_firms_20),
        mode='lines',
        line=dict(width=2, color='black'),
        showlegend=False,
    )
)

fig.add_trace(
    go.Scatter(
        x=largest_firms_20[x],
        y=largest_firms_20[y],
        mode='markers',
        marker=dict(size=16, color=largest_firms_20['composite_score'], colorscale='RdBu', showscale=True),
        showlegend=False,
        hovertext=largest_firms_20.index
    )
)

fig.add_trace(
    go.Scatter(
        x=largest_firms_20[x][:5],
        y=largest_firms_20[y][:5],
        mode='markers',
        marker=dict(size=8, color='yellow', opacity=0.8, symbol='star'),
        showlegend=False,
        hovertext=largest_firms_20.index
    )
)

# format figure to make it look nice
fig.update_layout(
    xaxis_title=x,
    yaxis_title=y,
    width=1000,
    height=500,
    template='plotly_white',
    font=dict(size=16),
    # yaxis_type='log',
)
fig.show()

### Volatility in profit (Net Combined ratio)

In [441]:
scr = largest_firms_16_20[['Firms_x','Net combined ratio']].droplevel(0, axis=1).T
scr.columns = scr.iloc[0]
scr = scr.drop('Firms_x')

fig = go.Figure()
x = pd.to_datetime(['2016', '2017', '2018', '2019', '2020'])

for col in largest_firms_20.index[5:]: # plot all large firms
    fig.add_trace(
        go.Scatter(
            x=x,
            y=scr[col],
            name=col,
            mode='lines',
            line=dict(width=1, color='grey')
        )
    )
for col in largest_firms_20.index[:5]: # plot identified firms
    fig.add_trace(
        go.Scatter(
            x=x,
            y=scr[col],
            name=col,
            mode='lines',
            line=dict(width=2, color='red'),
        )
    )

fig.add_trace(
    go.Scatter(
        x=x,
        y=[1]*len(scr),
        mode='lines',
        line=dict(color='black', width=4, dash='dash')
    )
)

fig.update_layout(
    xaxis_title='Year',
    # xaxis_range=[2016, 2020],
    yaxis_range=[-4, 5.5],
    yaxis_title='Net combined ratio',
    width=1200,
    height=500,
    template='plotly_white',
    font=dict(size=12),
    showlegend=False,
    xaxis_tickformat="%Y"
)

fig.show()

In [462]:
scr = largest_firms_16_20[['Firms_x','Total assets (£m)']].droplevel(0, axis=1).T
scr.columns = scr.iloc[0]
scr = scr.drop('Firms_x')

fig = go.Figure()
x = pd.to_datetime(['2016', '2017', '2018', '2019', '2020'])

for col in largest_firms_20.index[5:]: # plot all large firms
    fig.add_trace(
        go.Scatter(
            x=x,
            y=np.log(scr[col].astype(int) + 1),
            name=col,
            mode='lines',
            line=dict(width=1, color='grey')
        )
    )
for col in largest_firms_20.index[:5]: # plot identified firms
    fig.add_trace(
        go.Scatter(
            x=x,
            y=np.log(scr[col].astype(int) + 1),
            name=col,
            mode='lines',
            line=dict(width=2, color='red'),
        )
    )

fig.update_layout(
    xaxis_title='Year',
    # xaxis_range=[2016, 2020],
    # yaxis_range=[-4, 5.5],
    yaxis_title='Total assets (log, £m)',
    width=1200,
    height=500,
    template='plotly_white',
    font=dict(size=12),
    showlegend=False,
    xaxis_tickformat="%Y",
    # yaxis_type='log',
)

fig.show()

# Misc

## Sense check: are there any large firms of interest that had no assets during 2020?

## Not sure what the below code was trying to do..

In [ ]:
# Create an array of percentiles
percentiles_16_19 = np.array(range(10, 100, 10))

# Calculate deciles
lst_2016_19 = ['2016YE', '2017YE', '2018YE', '2019YE']
deciles_16_19 = np.percentile(df_merged['GWP (£m)'][lst_2016_19].sum(axis=1).values, percentiles_16_19)

# Create a bar plot of the decile counts
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=percentiles_16_19,
        y=deciles_16_19,
        name='GWP (£m)'
    )
)

fig.update_layout(
    title='Deciles of GWP (£m) 2016-2019',
    xaxis_title='Decile',
    yaxis_title='Amount (£m)'
)
fig.show()

####
largest_firms_16_19 = df_merged[df_merged['GWP (£m)'][lst_2016_19].sum(axis=1) > deciles_16_19[-1]]
largest_firms_16_19_insolvent = largest_firms_16_19[largest_firms_16_19['GWP (£m)']['2020YE'] == 0]
largest_firms_16_19_insolvent[['Firms_x', 'GWP (£m)', 'SCR coverage ratio', 'SCR (£m)', 'Total liabilities (£m)']]
####

####
for firm in largest_firms_16_19_insolvent['Firms_x']['Firms_x'].values:
    print(f"looking at {firm}")
    print("is firm in largest_firms_16_20?")
    if firm in largest_firms_16_20['Firms_x']['Firms_x'].values:
        print('Yes')
    else:
        print('No')
####

####
firm_179 = df_merged[df_merged['Firms_x']['Firms_x'] == 'Firm 179']
largest_firms_16_20 = pd.concat([largest_firms_16_20, firm_179], ignore_index=True)
####